# [모듈 3-2] Endpoint 테스트

## 0. 라이브러리 및 변수 설정

In [ ]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

## 1. 테스트 데이터셋 로딩

In [ ]:
import os
test_path= os.path.join('../data/dataset', 'test.csv')
test_df = pd.read_csv(test_path)
test_df.head()

## 2. 추론을 위한 Payload 생성
* Endpoint에 입력 될 수 있도록 CSV String 형태로 변환(Payload)
* 테스트 데이터에서 시범적으로 20개의 샘플만 변환

In [ ]:
def get_payload_list(test_df, label_col):
    '''
    CSV String 행태로 변경
    '''
    s = test_df.drop(columns=[label_col])
    s = s.to_csv(header=None, index=None)
    payload_list = s.splitlines()
    return payload_list
payload_list = get_payload_list(test_df[:5], label_col='fraud') # 샘플로 5개의 입력을 변환
y_test = test_df['fraud'].tolist()

print("payload_list: \n", payload_list)

## 3. 실시간 추론을 위한 Endpoint를 호출하여 추론 수행
* 엔드포인트가 InService 상태인 경우, 입력 데이터에 대한 추론 결과와 확률값을 제공함
* autogluon_serve.py에서 json 형태로 결과를 반환하기에, JSONDesirializer 사용

In [ ]:
import numpy as np
from sagemaker.deserializers import JSONDeserializer

json_deserializer = JSONDeserializer(accept='application/json')

endpoint_name = "ramp-test-staging"

predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session= sagemaker_session,
    deserializer = json_deserializer, # byte stream을 csv 형태로 변환하여 제공        
)

for payload in payload_list:
    result = predictor.predict(payload, initial_args = {"ContentType": "text/csv"})
    print(result)